In [56]:
import os
import pandas as pd

In [57]:
date_add = "202411-202412"

## Velox Commission

In [58]:
import pandas as pd

def filter_and_count_cancel_pairs(df):
    # Separate buy/sell and cancel orders for easier processing
    buy_sell_orders = df[df['Entry Type'].isin(['B', 'S'])]
    cancel_orders = df[df['Entry Type'].isin(['BXL', 'SXL'])]
    
    # Create dictionaries to store buy/sell orders and their corresponding cancels
    orders_dict = {}
    matched_pairs = []
    unmatched_rows = df.copy()

    # Iterate over buy/sell orders to populate the orders_dict
    for _, row in buy_sell_orders.iterrows():
        #print(row.name)
        key = (row['ACC#'], row['Symbol'], row['Entry Type'])
        if key not in orders_dict:
            orders_dict[key] = []
        orders_dict[key].append((row['Qty'], row['Gross Amount'], row['Trade Date'], row.name))

    # Iterate over cancel orders to find matching buy/sell orders
    for _, row in cancel_orders.iterrows():
        key = (row['ACC#'], row['Symbol'], 'B' if row['Entry Type'] == 'BXL' else 'S')
        if key in orders_dict:
            for i, (shares, amount, date, index) in enumerate(orders_dict[key]):
                # Check if the shares and amount are additive inverses
                if shares == -row['Qty'] and amount == -row['Gross Amount']:
                    # Found a matching pair, remove it from orders_dict
                    matched_pairs.append(df.loc[[index, row.name]])
                    orders_dict[key].pop(i) 
                    unmatched_rows.drop(index=[index, row.name], inplace=True)
                    break

    # Create a dataframe containing all matched pairs
    matched_pairs_df = pd.concat(matched_pairs) if matched_pairs else pd.DataFrame(columns=df.columns)
    
    # Return the count of pairs, the matched pairs dataframe, and the dataframe without the pairs
    cancel_count = len(matched_pairs) // 2
    return cancel_count, matched_pairs_df, unmatched_rows

In [59]:
list = ["93", "98", "85"]
out_dict = {}

for rep in list:
    velox = pd.read_excel(f"namelist/commission/{date_add}/velox_comm/velox_{rep}.xlsx")[[
        "ACC#", "Trade Date", "Settle Date", "Entry Type", "Symbol", "Qty", "Price", "Gross Amount", "Net Amount", "Comm"
    ]]
    velox = velox[velox["Comm"]!=0]
    count, pairs, unmatched = filter_and_count_cancel_pairs(velox)
    
    # for any regular trade, the commission deduct is $5, for any pair it is $10
    unmatched["Comm Deduct"] = 5
    pairs["Comm Deduct"] = 10
    unmatched["Net Commission"] = pd.NA
    
    # Calculate commission
    pairs_single = pairs[pairs["Entry Type"].isin(["SXL","BXL"])] # pairs commission only goes once
    unmatched = pd.concat([unmatched, pairs_single], axis=0)
    commission = sum(unmatched["Comm"])
    deduct = sum(unmatched["Comm Deduct"])
    net_comm = abs((commission+deduct)*0.6)

    # Put the 3 values to the dataframe
    unmatched = unmatched.reset_index()
    pairs_single = pairs_single.reset_index()
    pairs = pairs.reset_index()
    tail = len(unmatched)
    for i,j in [(commission, "Comm"), (deduct, "Comm Deduct"), (net_comm, "Net Commission")]:
        #print(i,j)
        unmatched.loc[tail, j] = i
        unmatched.fillna("")

    out_dict[rep] = [count, pairs, unmatched]

    pairs.to_csv(f"namelist/commission/{date_add}/velox_comm/velox_comm_filtered/velox_{rep}_pairs.csv")
    unmatched.to_csv(f"namelist/commission/{date_add}/velox_comm/velox_comm_filtered/velox_{rep}_unmatched.csv")


C:\Users\info\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\info\AppData\Local\Temp\ipykernel_45244\3470408429.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  unmatched = pd.concat([unmatched, pairs_single], axis=0)
C:\Users\info\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workboo

## Curv Commission

In [60]:
def add_at_tail(df, name_val):
    tail = len(df)
    for name, value in name_val:
        df[name] = pd.NA
        df.loc[tail, name] = value
    return df

def clean_cash(value):
    # Remove dollar signs and commas
    value = value.replace('$', '').replace(',', '')
    # Handle parentheses for negative values
    if '(' in value and ')' in str(value):
        value = '-' + value.replace('(', '').replace(')', '')
    return value

In [61]:
curv_rep_all = ["85", "93", "98"]
special_rep = ["85"]
special_acc = ["AC9900022", "AC9900182"]

col_names = [
    "Account No", "Account Name", "Rep",
    "Side", "Eff Trade Date", "Symbol", "Trade Date", "Qty", "Price", "Gross Amt", "Trns Id",
    "Status", "Fees", "Net Amt", "Commission", "Contra Correspondent"
]

In [62]:
# Path to the folder containing the CSV files
folder_path = f"namelist/commission/{date_add}/curv_comm"

# List all CSV files in the folder
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

csv_files

['curv_85.csv', 'curv_93.csv', 'curv_98.csv']

In [63]:
# Concatenate all CSV files
dataframes = [pd.read_csv(os.path.join(folder_path, file)) for file in csv_files]
combined_df = pd.concat(dataframes, ignore_index=True)

# Some data cleaning and format casting
combined_df = combined_df[col_names]
combined_df["Rep"] = combined_df["Rep"].fillna(0).astype(int).astype(str)
combined_df["Qty"] = combined_df["Qty"].str.replace(",","").fillna(0).astype(int)
combined_df['Commission'] = combined_df['Commission'].astype("str").apply(clean_cash).astype("float")
combined_df['Gross Amt'] = combined_df['Gross Amt'].astype("str").apply(clean_cash)

# kick out all ACSS Contra Correspondent, and delete all records with Quantity == 0
combined_df = combined_df[combined_df["Contra Correspondent"]!="ACSS"]
combined_df = combined_df[combined_df["Qty"]!=0]

In [66]:
# filter with Rep number
out_dict = {}
for rep in curv_rep_all:
    curv_rep = combined_df[combined_df["Rep"]==rep]
    if rep not in special_rep:
        out_dict[rep] = curv_rep
    else:
        special_acc_df = curv_rep[curv_rep["Account No"].isin(special_acc)]
        other_acc_df = curv_rep[~curv_rep["Account No"].isin(special_acc)]
        out_dict[rep] = {
            "special": special_acc_df,
            "other": other_acc_df
        }


In [67]:
import pandas as pd

def filter_and_count_cancel_pairs_curv(df):
    # Separate orders based on their status
    cancel_orders = df[df['Status'] == 'Cancel']
    executed_cxld_orders = df[df['Status'] == 'Executed CXLD']
    correct_cxld_orders = df[df['Status'] == 'Correct CXLD']
    
    # Create dictionaries to store orders and their corresponding cancels
    orders_dict = {}
    matched_pairs = []
    unmatched_rows = df.copy()

    # Iterate over executed and correct canceled orders to populate the orders_dict
    for _, row in pd.concat([executed_cxld_orders, correct_cxld_orders]).iterrows():
        key = (row['Account No'], row['Symbol'], row['Status'])
        if key not in orders_dict:
            orders_dict[key] = []
        orders_dict[key].append((row['Qty'], row['Gross Amt'], row['Trade Date'], row.name))

    # Iterate over cancel orders to find matching executed or correct canceled orders
    for _, row in cancel_orders.iterrows():
        key_executed = (row['Account No'], row['Symbol'], 'Executed CXLD')
        key_correct = (row['Account No'], row['Symbol'], 'Correct CXLD')
        
        # Check for matching pairs with Executed CXLD or Correct CXLD
        for key in [key_executed, key_correct]:
            if key in orders_dict:
                for i, (shares, amount, date, index) in enumerate(orders_dict[key]):
                    # Check if the shares and amount are additive inverses
                    if shares == -row['Qty'] and amount == -row['Gross Amt']:
                        # Found a matching pair, remove it from orders_dict
                        matched_pairs.append(df.loc[[index, row.name]])
                        orders_dict[key].pop(i)
                        if row.name==10310:
                            print(row)
                            print(row.name)
                            print(index)
                        #unmatched_rows.drop(index=[index, row.name], inplace=True)
                        break

    # Create a dataframe containing all matched pairs
    matched_pairs_df = pd.concat(matched_pairs) if matched_pairs else pd.DataFrame(columns=df.columns)
    
    # Return the count of pairs, the matched pairs dataframe, and the dataframe without the pairs
    cancel_count = len(matched_pairs) // 2
    return cancel_count, matched_pairs_df, unmatched_rows

In [68]:
def curv_filter(df, pct):
    if df['Commission'].dtype != float:
        df['Commission'] = df['Commission'].apply(clean_cash)
    if df['Gross Amt'].dtype != float:
        df['Gross Amt'] = df['Gross Amt'].apply(clean_cash)

    count, pairs, unmatched = filter_and_count_cancel_pairs_curv(df)
    print(count, len(pairs))
    total_shares = sum(df['Qty'])
    gross_comm = sum(df['Commission'])

    print("3 numbers")
    print(gross_comm, count*50, total_shares*0.007)
    print("down")
    net_comm = (abs(gross_comm) - count*50 - abs(total_shares)*0.007)*pct

    df.loc[len(df),"Contra Correspondent"] = gross_comm
    df = add_at_tail(df, [
        ("Pair Cancelling Deduction", count*50),
        ("To Curv and Spirit", total_shares*0.007),
        ("Net Commission", net_comm)
    ])

    #df = add_at_tail(df, count*50, "Pair Cancelling Deduction")
    #df = add_at_tail(df, total_shares*0.007, "To Curv and Spirit")
    #df = add_at_tail(df, net_comm, "Net Commission")

    return df

In [69]:
for i, key in enumerate(out_dict):
    data = out_dict[key]
    print(key)
    if type(data) == dict: # special
        special = data['special']
        special = curv_filter(special, 0.6)
        special.to_csv(f'{folder_path}/curv_comm_filtered/{key}_comm_special.csv')

        other = data['other']
        other = curv_filter(other, 0.1)
        other.to_csv(f'{folder_path}/curv_comm_filtered/{key}_comm_other.csv')
    else:
        data = curv_filter(data, 0.6)
        data.to_csv(f'{folder_path}/curv_comm_filtered/{key}_comm.csv')


85
0 0
3 numbers
0 0 0.0
donw
0 0
3 numbers
-2783.47 0 -3252.732
donw
93
0 0
3 numbers
0 0 0.0
donw
98
0 0
3 numbers
0 0 0.0
donw


C:\Users\info\AppData\Local\Temp\ipykernel_45244\678064434.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[len(df),"Contra Correspondent"] = gross_comm
C:\Users\info\AppData\Local\Temp\ipykernel_45244\678064434.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[len(df),"Contra Correspondent"] = gross_comm


: 

In [406]:
temp = pd.read_csv(f"{folder_path}/Activity_Oct 2024.csv")
temp = temp[temp['Account No']=="AC9900174"][col_names]

C:\Users\info\AppData\Local\Temp\ipykernel_7596\1276636170.py:1: DtypeWarning: Columns (28,48) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(f"{folder_path}/Activity_Oct 2024.csv")


In [407]:
temp.to_csv(f"{folder_path}/HuaLi.csv")